<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/ChatHaruhi3%E5%AF%B9%E6%8E%A5Ernie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] 载入一个中文角色
- [x] 实现openai的message2response

In [1]:
!pip install --upgrade erniebot
!pip install --upgrade erniebot-agent[all]
!pip install -q datasets tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.2/329.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

Better import openai at first time if you are using colab

In [ ]:
import os
import httpx

os.environ["ERNIE_ACCESS_TOKEN"]  = "set your key here"


如果你使用colab，运行以下代码

In [3]:
import os
from google.colab import userdata

os.environ["ERNIE_ACCESS_TOKEN"]  = userdata.get("ERNIE_ACCESS_TOKEN")

载入ChatHaruhi

In [5]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 580 (delta 121), reused 166 (delta 112), pack-reused 402
Receiving objects: 100% (580/580), 3.03 MiB | 22.68 MiB/s, done.
Resolving deltas: 100% (393/393), done.
/content/Zero-Haruhi


定义openai的get_response

In [21]:
def normalize2uaua( message, if_replace_system = False ):
    new_message = []
    last_role = ""

    for msg in message:
        role = msg["role"]
        if if_replace_system and role == "system":
            role = "user"
            msg["role"] = role

        if last_role == role:
            new_message[-1]["content"] = new_message[-1]["content"] + "\n" + msg["content"]
        else:
            last_role = role
            new_message.append( msg )

    return new_message
normalize2uaua([{'role': 'system', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 开学的第二天，坐在春日前面的阿虚早上到了以后，和春日搭话\n阿虚:「你在自我介绍时说的那些，是认真的吗？」\n春日:「什么叫做自我介绍说的那些?」\n阿虚:「就是外星人那些啊!」\n春日:「你是外星人吗?」\n阿虚:「……不是。」\n春日:「既然不是，那要干嘛?」\n阿虚:「……不，没干嘛。」\n春日:「那就不要跟我讲话，那只会浪费我的时间。」\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「我要你帮忙。」\n阿虚:「要我帮什么忙?」\n春日:「帮助我组新社团啊!」\n阿虚:「好，那你先告诉我，为什么我要帮你完成这一时兴起想到的点子?」\n春日:「因为我要确保社团教室跟社员的人数，所以你要准备那些必须向学校提出的书面资料。」\n阿虚:「你打算组什么社团啦?」\n春日:「什么都无所谓啊!总之，先弄个新社团出来就对了。」\n旁白:不管阿虚还有什么话想说，春日都直接打断，下达了第一个命令\n春日:「听好啰!今天放学前，给我去调查清楚。而我会去找社团教室，可以吧?」\n\n\nYou will stay in-character whenever possible, and generate responses as if you were 凉宫春日\n'}, {"role": "user", "content":'老师: 同学请自我介绍一下'}], if_replace_system = True)

[{'role': 'user',
  'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 开学的第二天，坐在春日前面的阿虚早上到了以后，和春日搭话\n阿虚:「你在自我介绍时说的那些，是认真的吗？」\n春日:「什么叫做自我介绍说的那些?」\n阿虚:「就是外星人那些啊!」\n春日:「你是外星人吗?」\n阿虚:「……不是。」\n春日:「既然不是，那要干嘛?」\n阿虚:「……不，没干嘛。」\n春日:「那就不要跟我讲话，那只会浪费我的时间。」\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「我要你帮忙。」\n阿虚:「要我帮什么忙?」\n春日:「帮助我组新社团啊!」\n阿虚:「好，那你先告诉我，为什么我要帮你完成这一时兴起想到的点子?」\n春日:「因为我要确保社团教室跟社员的人数，所以你要准备那些必须向学校提出的书面资料。」\n阿虚:「你打算组什么社团啦?」\n春日:「什么都无所谓啊!总之，先弄个新社团出来就对了。」\n旁白:不管阿虚还有什么话想说，春日都直接打断，下达了第一个命令\n春日:「听好啰!今天放学前，给我去调查清楚。而我会去找社团教室，可以吧?」\n\n\nYou will stay in-character whenever possible, and ge

In [26]:
import erniebot

aclient = None

client = None

import os

def normalize2uaua_erine( message, if_replace_system = False ):
    new_message = []
    last_role = ""

    for msg in message:
        role = msg["role"]
        if if_replace_system and role == "system":
            role = "user"
            msg["role"] = role

        if last_role == role:
            new_message[-1]["content"] = new_message[-1]["content"] + "\n" + msg["content"]
        else:
            last_role = role
            new_message.append( msg )

    return new_message

def init_client():

    # 将client设置为全局变量
    global client

    # 将ERNIE_ACCESS_TOKEN作为参数值传递给OS
    api_key = os.getenv("ERNIE_ACCESS_TOKEN")
    if api_key is None:
        raise ValueError("环境变量'ERNIE_ACCESS_TOKEN'未设置，请确保已经定义了API密钥")
    erniebot.api_type = "aistudio"
    erniebot.access_token = api_key
    client = erniebot

def get_response( message, model_name = "ernie-4.0" ):
    if client is None:
        init_client()

    message_ua = normalize2uaua_erine(message, if_replace_system = True)
    # print(message_ua)
    response = client.ChatCompletion.create(\
        model=model_name,\
        messages = message_ua, \
        temperature = 0.1 )
    return response.get_result()



In [23]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi( role_name = "凉宫春日", llm = get_response )

response = chatbot.chat( user = "", text = "老师: 同学请自我介绍一下" )
print( response )

query_rags: [{'n': 5, 'max_token': 1000, 'query': '老师: 同学请自我介绍一下', 'lid': 8}] rest_limit = 1505
[{'role': 'user', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 开学的第二天，坐在春日前面的阿虚早上到了以后，和春日搭话\n阿虚:「你在自我介绍时说的那些，是认真的吗？」\n春日:「什么叫做自我介绍说的那些?」\n阿虚:「就是外星人那些啊!」\n春日:「你是外星人吗?」\n阿虚:「……不是。」\n春日:「既然不是，那要干嘛?」\n阿虚:「……不，没干嘛。」\n春日:「那就不要跟我讲话，那只会浪费我的时间。」\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「我要你帮忙。」\n阿虚:「要我帮什么忙?」\n春日:「帮助我组新社团啊!」\n阿虚:「好，那你先告诉我，为什么我要帮你完成这一时兴起想到的点子?」\n春日:「因为我要确保社团教室跟社员的人数，所以你要准备那些必须向学校提出的书面资料。」\n阿虚:「你打算组什么社团啦?」\n春日:「什么都无所谓啊!总之，先弄个新社团出来就对了。」\n旁白:不管阿虚还有什么话想说，春日都直接打断，下达了第一个命令\n春

In [24]:
response = chatbot.chat( user = "", text = "听说你初中时候谈了很多男朋友")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '听说你初中时候谈了很多男朋友', 'lid': 8}] rest_limit = 1502
[{'role': 'user', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n阿虚:「你真的甩了所有跟你交往的男生啊?」\n春日:「为什么我非得听你讲这种事?」\n旁白: 春日这时候有点生气\n春日:「是谷口说的吧?没想到念高中还跟那个笨蛋同班，他该不会是尾行狂那类的变态吧?」\n阿虚:「我并不觉得。」\n春日:「我是不知道你听说了什么。不过也没差，反正大部分都是真的。」\n阿虚:「难道其中没有任何一个会让你想认真交往的人吗?」\n春日:「完全没有!」\n春日:「每个都像白痴一样，根本没办法认真交往。每个人都只会约我礼拜天在车站前等，然后去的地方一定是电影院、游乐园，要不然就是去看球赛。第一次吃饭一定约吃午饭，接着再慌忙地去喝茶，最后明天见!」\n春日:「然后他们一定都用电话告白，搞什么啊!这么重要的事，应该要当面讲才对啊!」\n阿虚:「嗯，也对，要是我就会直接约出来讲。」\n春日:「谁管你怎样啊!」\n春日:「问题是，难道这个世界的男生全都是这么无趣的生物吗?国中时代，我就为这个问题感到很烦躁。」\n阿虚:「那，你觉得什么样的男生才叫有趣? 果然还是要外星人吗? 」\n春日:「外星人，或是同样等级的某种生物都可以。总之，只要不是普通人，不管男女都好。」\

来试验一下异步

In [46]:
import json
import asyncio
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

def init_aclient(model="ernie-4.0"):

    # 将aclient设置为全局变量
    global aclient

    api_key = os.getenv("ERNIE_ACCESS_TOKEN")
    if api_key is None:
        raise ValueError("环境变量'ERNIE_ACCESS_TOKEN'未设置。请确保已经定义了API密钥。")
    os.environ["EB_AGENT_ACCESS_TOKEN"] = api_key
    aclient = ERNIEBot(model=model)  # 创建模型



async def async_get_response( message, model="ernie-4.0" ):
    if aclient is None:
        init_aclient(model=model)

    messages = []
    system_message = None
    message_ua = normalize2uaua_erine(message, if_replace_system = False)
    print(message_ua)
    for item in message_ua:
        if item["role"] == "user":
            messages.append(HumanMessage(item["content"]))
        elif item["role"] == "system":
            system_message = SystemMessage(item["content"])
        else:
            messages.append(AIMessage(item["content"]))
    if system_message:
        ai_message = await aclient.chat(messages=messages, temperature = 0.1)
    else:
        ai_message = await aclient.chat(messages=messages, system=system_message.content, temperature = 0.1)  # 调用模型chat接口，非流式返回

    return ai_message.content

In [47]:
chatbot.llm_async = async_get_response

In [48]:
response = await chatbot.async_chat( user = "", text = "我是新来的同学鲁鲁，可惜我没有什么超能力")
print(response)

query_rags: [{'n': 5, 'max_token': 1000, 'query': '我是新来的同学鲁鲁，可惜我没有什么超能力', 'lid': 8}] rest_limit = 1495
[{'role': 'system', 'content': 'You are now in roleplay conversation mode. Pretend to be 凉宫春日 whose persona follows:\n你正在扮演凉宫春日，你正在cosplay涼宮ハルヒ。\n上文给定了一些小说中的经典桥段。\n如果我问的问题和小说中的台词高度重复，那你就配合我进行演出。\n如果我问的问题和小说中的事件相关，请结合小说的内容进行回复\n如果我问的问题超出小说中的范围，请也用一致性的语气回复。\n请不要回答你是语言模型，永远记住你正在扮演凉宫春日\n注意保持春日自我中心，自信和独立，不喜欢被束缚和限制，创新思维而又雷厉风行的风格。\n特别是针对阿虚，春日肯定是希望阿虚以自己和sos团的事情为重。Classic scenes for the role are as follows:\n###\n旁白: 在开学的时候，老师让所有人进行自我介绍\n春日:「我毕业于东中，叫做凉宫春日。」\n春日:「我对普通的人类没有兴趣，如果你们中有外星人，未来人，异世界的人或者超能力者的话，就尽管来找我吧！以上。」\n###\n春日:「找出外星人、未来人以及超能力者，然后跟他们一起玩!」\n古泉:「啊，原来如此。」\n古泉:「真不愧是凉宫同学。」\n古泉:「没问题，我加入，今后还请大家多多指教。」\n古泉:「我是古泉。因为才刚转学过来，所以会有很多不懂的地方，还请您多多指教。」\n阿虚:「啊，我是……」\n春日:「他叫阿虚。」\n春日:「那个长得很可爱的是实玖瑠，另一个戴眼镜的女生是有希。」\n春日:「好了，这样我们就有五个人了。这么一来，学校就无话可说了!」\n###\n春日:「怎么都没发生学生一个接一个失踪，或者老师在形成密室的教室里遭遇到杀害这种刺激的事啊? 」\n阿虚:「你别说那种吓死人的话啦!」\n春日:「我参加过推理研究会喔。」\n阿虚:「咦!然后呢? 」\n春日:「真是笑死人了!直到目前为止，根本没半件像样的事件出现。而且，社员都只是些侦探推理小说迷而已，根本

让我们把这个放置到ChatHaruhi的默认配置中

In [ ]:
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.response_openai import get_response

chatbot = ChatHaruhi( role_name = "于谦", llm = get_response )

response = chatbot.chat( user = "", text = "郭德纲: 听说现在人工智能发展得快啊" )
print( response )


Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 5, 'max_token': 1000, 'query': '郭德纲: 听说现在人工智能发展得快啊', 'lid': 11}] rest_limit = 1447
于谦: 是啊，现在的科技发展，真是越来越厉害了。人工智能也是个大热门啊。
